# Citation

```
@misc{ToD-calibrate-2024,
  author = {Matthew Yucha},
  title = {TextEvolve Calibrate},
  year = {2024},
  month = sep,
  url = {https://github.com/theobjectivedad/research/blob/main/calibrate/calibrate.ipynb},
  note = {Jupyter Notebook, Contact: theobjectivedad@gmail.com}
}
```

# TextEvolve Calibrate


*Calibrate* is a semi-supervised automatic prompt optimization (APO) method where a large language model (LLM) is used to automatically improve the performance of a prompt given a calibration data set. Similar to [Evaluate](https://github.com/theobjectivedad/research/blob/main/evaluate/Evaluate%20Demo.ipynb).

*Calibrate* is an enhancement to Prompt Optimization with Textual Gradients (ProTeGi) [[arXiv:2305.03495](https://arxiv.org/abs/2305.03495)]. From the paper, ProTeGi refers to a novel APO method that proposes using a gradient descent-style algorithm to automatically optimize an LLM prompt against a training data set. While ProTeGi demonstrated impressive accuracy gains of up to 31% across three benchmark tasks, including jailbreak detection, there two significant limitations:

1. Classification only: ProTeGi is limited to optimizing classification tasks (ex: jailbreak attempt/no jailbreak attempt)
2. Serial beam selection: ProTeGi's proposed beam selection methods, *UCB Bandits*, *UCB-E*, *Successive Rejects*, and *Successive Halving* need to be computed serially making them challenging to scale.
3. On its own, ProTeGi implements a supervised optimization methodology and requires data labeling.

*Calibrate* addresses these limitations by replacing *UCB Bandits* beam selection with a batch capable LLM-based method. Additionally, *Evaluate* is used as the metric function $m$ which allows assigning a continuous numeric score to candidates during the selection process. 

*Calibrate* and *Evaluate* are part of the larger TextEvolve service suite. TextEvolve is a collection of integrated generative AI services focused on the guided and automated reasoning over text-based data. One of the main goals of TextEvolve is to remove the most labor-intensive components of building AI systems including: human evaluation, prompt engineering, and data set labeling. 

```text
    ┌────────────────────┐  ┌────────────────────┐
    │                    │  │                    │
    │      Calibrate     │  │      Innovate      │
    │                    │  │                    │
    └────────────────────┘  └────────────────────┘
    ┌────────────────────────────────────────────┐
    │                                            │
    │                  Evaluate                  │
    │                                            │
    └────────────────────────────────────────────┘
    ┌────────────────────────────────────────────┐
    │                                            │
    │                  Generate                  │
    │                                            │
    └────────────────────────────────────────────┘
```

As shown in the diagram above, each TextEvolve service builds from the others.

**Generate**: Automatically generates agent profiles for *Evaluate* given an input task.

**Evaluate**: Leverages a multi-agent debate (MAD) methodology to generate a numeric score given an input and set of outputs.

**Calibrate**: A semi-supervised automatic prompt optimization (APO) method where a large language model (LLM) is used to automatically improve the performance of a prompt given a calibration data set.

**Innovate**: An unsupervised method of iteratively improving the quality of an input. Like *Calibrate*, it leverages *Evaluate* to compute scores and "gradients" to guide the improvement process.

## Method

In every round $r$, expanded prompt template candidates $\mathcal{C}$ are computed by applying the  $\operatorname{expand}$ function over all elements of our beam $\mathcal{B}$. 

$$
C = \left\{ \operatorname{expand}\left(t': b, \mathcal{D}_{cal}\right) \mid b \in \mathcal{B} \right\}
$$

During the expansion process, a mini-batch of samples are randomly selected from $\mathcal{D}_{cal}$ and used to batch generate a response from $\operatorname{LLM}$: 

$$
\operatorname{LLM}\left(\text{generations}: \text{minibatch} \right)
$$

Next, "gradients" $\mathcal{g}$ are computed by invoking *Evaluate* where $x$ is the current prompt template in $b$ and $\mathcal{y}$ contains the generations from the previous step. The actual values in $\mathcal{g}$ are natural language instructions on how $b$ could be changed to achieve a better score w.r.t. the template values sampled from $\mathcal{D}_{cal}$ in the current mini-batch:

$$
\operatorname{LLM}_{\nabla}\left(\mathcal{g} : x, \mathcal{y}\right)
$$

The last step is where we will actually generate new prompt templates given our current beam $b$, and gradients $\mathcal{g}$:

$$
\operatorname{LLM}_{\sigma}\left(t'_b: b, \mathcal{g} \right)
$$

At this point expanded prompt template candidates $\mathcal{C}$ have been derived and the *Calibrate* fast selection process is executed:

$$
\operatorname{select}_{\text{fast}}\left( \mathcal{B'}: \mathcal{B}, \mathcal{C}, \mathcal{D}_{cal} \right)
$$

Here, $\operatorname{select}_{\text{fast}}$ will again sample a mini-batch from $\mathcal{D}_{cal}$, generate outputs via $\operatorname{LLM}$  and compute numeric scores via *Evaluate*. The implementation will execute batch inferencing operations on all LLM calls, including *Evaluate* to achieve scalability. New beams $\mathcal{B}$ will be selected from the highest scoring prompt templates and the next round will begin.


## Simulation

This simulation shows a demonstration calibration run.


Suppose we have a simple prompt template $t_0$:

```
Write a {document} about {topic}
```

We would like to automatically calibrate this prompt template against the following set of data $D_{\text{cal}}$, observe that each element contains natural language values for $t_0$: 

```json
[
    {"document": "report", "topic": "AI"},
    {"document": "essay", "topic": "philosophy"},
    {"document": "article", "topic": "technology"},
    {"document": "post", "topic": "economics"},
    {"document": "review", "topic": "politics"},
    {"document": "paper", "topic": "history"},
    {"document": "guide", "topic": "science"},
    {"document": "letter", "topic": "art"},
    {"document": "memo", "topic": "education"},
    {"document": "summary", "topic": "health"},
    {"document": "proposal", "topic": "business"},
    {"document": "manual", "topic": "literature"},
    {"document": "story", "topic": "environment"},
    {"document": "script", "topic": "culture"},
    {"document": "novel", "topic": "society"},
    {"document": "blog", "topic": "media"},
    {"document": "journal", "topic": "sports"},
    {"document": "thesis", "topic": "entertainment"},
    {"document": "dissertation", "topic": "food"},
    {"document": "book", "topic": "travel"}
]
```

Given $t_0$ and $\mathcal{D}_{\text{cal}}$, we initialize *Calibrate* with the following hyper-parameters:


| Parameter | Value | Description |
| --------- | ----- | ----------- |
| ``b`` | 4 | Beam width, controls how many beams to explore for each search level $r$ |
| ``r`` | 5 | Search depth, controls how many search iterations are performed |
| ``minibatch_exp_len`` | 5 | Number of calibration data samples used during expansion |
| ``minibatch_sel_len`` | 6 | Number of calibration data samples for the scoring step during selection |
| ``grad_depth`` | 10 | Number of suggestions the LLM generates when constructing gradients |
| ``grad_debate_rounds`` | 1 | Number of *Evaluate* debate rounds to consider when generating gradients. |
| ``mc_successors`` | 5 | Number of monte-carlo successors generated from the improved template. |

Next, we execute the simulation which modifies *Evaluate* to always increments $\mathcal{B}$ by $+0.200$ when $\mathcal{B}_i=2$, and decrements $\mathcal{B}$ by the same amount for all other beams: 

```text
2024-09-19 13:41:21 INFO r=01 expand(B=00): len(t')=05, len(C)=05
2024-09-19 13:41:21 INFO r=01 select[fast]: len(B')=4, new beams=4, len(B)=4, B mean=5.0375 (4.9833, 5.2000, 4.9833, 4.9833)

2024-09-19 13:41:21 INFO r=02 expand(B=00): len(t')=05, len(C)=05
2024-09-19 13:41:21 INFO r=02 expand(B=01): len(t')=05, len(C)=10
2024-09-19 13:41:21 INFO r=02 expand(B=02): len(t')=05, len(C)=15
2024-09-19 13:41:21 INFO r=02 expand(B=03): len(t')=05, len(C)=20
2024-09-19 13:41:22 INFO r=02 select[fast]: len(B')=4, new beams=1, len(B)=4, B mean=5.0750 (4.9667, 5.4000, 4.9667, 4.9667)

2024-09-19 13:41:22 INFO r=03 expand(B=00): len(t')=05, len(C)=05
2024-09-19 13:41:23 INFO r=03 expand(B=01): len(t')=05, len(C)=10
2024-09-19 13:41:23 INFO r=03 expand(B=02): len(t')=05, len(C)=15
2024-09-19 13:41:23 INFO r=03 expand(B=03): len(t')=05, len(C)=20
2024-09-19 13:41:24 INFO r=03 select[fast]: len(B')=4, new beams=1, len(B)=4, B mean=5.1125 (4.9500, 5.6000, 4.9500, 4.9500)

2024-09-19 13:41:24 INFO r=04 expand(B=00): len(t')=05, len(C)=05
2024-09-19 13:41:24 INFO r=04 expand(B=01): len(t')=05, len(C)=10
2024-09-19 13:41:24 INFO r=04 expand(B=02): len(t')=05, len(C)=15
2024-09-19 13:41:24 INFO r=04 expand(B=03): len(t')=05, len(C)=20
2024-09-19 13:41:25 INFO r=04 select[fast]: len(B')=4, new beams=1, len(B)=4, B mean=5.1500 (4.9333, 5.8000, 4.9333, 4.9333)

2024-09-19 13:41:25 INFO r=05 expand(B=00): len(t')=05, len(C)=05
2024-09-19 13:41:25 INFO r=05 expand(B=01): len(t')=05, len(C)=10
2024-09-19 13:41:25 INFO r=05 expand(B=02): len(t')=05, len(C)=15
2024-09-19 13:41:26 INFO r=05 expand(B=03): len(t')=05, len(C)=20
2024-09-19 13:41:26 INFO r=05 select[fast]: len(B')=4, new beams=1, len(B)=4, B mean=5.1875 (4.9167, 6.0000, 4.9167, 4.9167)
```

### Inferencing Volume

This table aggregates LLM operation statistics from the simulation run.

| LLM Operation                                    | Total | Success | Error | Request Tokens | Response Tokens | Total Tokens |
| :----------------------------------------------- | ----: | ------: | ----: | -------------: | --------------: | -----------: |
| $\operatorname{E}$, Evaluate turn                |   612 |     612 |     0 |        1,089,352 |          516,764 |      1,606,116 |
| $\operatorname{LLM_{\text{gen}, \text{select}}}$ |   510 |     510 |     0 |         100,638 |          104,733 |       205,371 |
| $\operatorname{LLM}_{\text{gen}, \text{expand}}$ |    85 |      85 |     0 |          16,617 |           17,492 |        34,109 |
| $\operatorname{LLM}_{\sigma}$                    |    85 |      85 |     0 |         215,315 |           17,100 |       232,415 |
| $\operatorname{LLM}_{\nabla}$                    |    17 |      17 |     0 |           6,978 |            3,414 |        10,392 |



### Costs

This table aggregates estimated costs across OpenAI models from the simulation run. Prices are current as of 2024-08-17.

| LLM Name   | Total Cost | Request Cost | Response Cost | Request Tokens | Response Tokens | Total Tokens | Generation Count |
| ---------- | ---------- | ------------ | ------------- | -------------- | --------------- | ------------ | ---------------- |
| gpt4o      | \$10.17    | \$3.57       | \$6.60        | 1,428,900      | 659,503         | 2,088,403    | 1,309            |
| gpt4o-mini | \$0.61     | \$0.21       | \$0.40        | 1,428,900      | 659,503         | 2,088,403    | 1,309            |




## Future Work

1. From the ProTeGi paper, *UCB Bandits* was the best performing beam selection method and despite it's scalability shortcomings may improve the effectiveness of *Calibrate*.

2. The fast selection method implemented by *Calibrate* could be improved significantly by testing more mini-batches of calibration data. Additional testing is needed to confirm the hypothesis that by scoring more calibration data during select, performance will be significantly improved (similar to a validation operation during deep neural network training). Additionally, with enough $\mathcal{D}_{\text{cal}}$ examples, there may be additional performance improvements realized by splitting $\mathcal{D}_{\text{cal}}$ into a calibration and validation data set to help prevent over-fitting prompt templates to calibration data that doesn't fully represent the overall task of the prompt.

3. Automatic hyper-parameter optimization; this will be needed to make *Calibrate* available to the everyman.

4. Additional research is needed to determine to what extent other beam selection methods such as *UCB Bandits* improve the overall performance of *Calibrate*.

5. Large context support could be implemented via a map-reduce summarization (I think).

6. While basic prompts were provided for $\operatorname{LLM}_{\sigma}$ and $\operatorname{LLM}_{\nabla}$, specific optimizations described in [[arXiv:2406.06608](https://arxiv.org/abs/2406.06608)] would likely improve performance.

7. Integration with experiment tracking tools such as [WanDB.ai](https://wandb.ai/)

8. Early stopping mechanics can be improved by checking a score $s$ tolerance, ex $\overline{s} \pm x$

## Appendix A: Code

This section contains an executable demonstration of *Evaluate* and *Calibrate*.

### Dependencies

The following code blocks contain dependencies needed for this demo. These include a simple LLM simulation class as well as an implementation of [Evaluate](https://www.theobjectivedad.com/pub/20240827-textevolve-evaluate/index.html). Please execute these cells first before running the demo.

In [1]:
"""LLM Simulator for offline development and testing"""

import json
import logging
import random
import re
import threading
import time
from abc import ABC, abstractmethod
from decimal import ROUND_HALF_UP, Decimal
from typing import Any, Dict, List, Literal, Tuple, Union

import tiktoken
from faker import Faker
from pydantic import BaseModel, Field


class LLMSimulatorBase(ABC):
    """
    Simulates calling a Language Model (LLM) by generating a list of simulated
    responses from one or more rendered prompts.

    This class is a very loosely based on langchain_core.langchain_models.llms.BaseLLM
    """

    @abstractmethod
    def __call__(
        self,
        prompts: Union[List[str], str],
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:
        """
        Abstract method to generate simulated LLM responses.

        Args:
            prompts (Union[List[str], str]): Fully rendered prompt(s) that would be
                passed to the LLM.
            n (int, optional): Number of responses to generate for each prompt. Defaults
                to 1.
            prefix (str, optional): Prefix to add to each generated response. Defaults
                to "GEN".
            append (str, optional): String to append at the end of each generated
                response. Defaults to None.
            min_tokens (int, optional): Minimum number of tokens in each simulated LLM
                response. Defaults to None.
            max_tokens (int, optional): Maximum number of tokens in each simulated LLM
                response. Defaults to None.
            **kwargs: Additional arguments, intended use is to document / capture
                additional model parameters passed to inferencing function

        Returns:
            List[str | None]: A list of generated responses, each corresponding to a
                prompt. When a respnse contains None, it represents a LLM inferencing
                error.
        """

    def invoke(
        self,
        prompt: str,
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:
        return self(
            prompt,
            n=n,
            prefix=prefix,
            append=append,
            min_tokens=min_tokens,
            max_tokens=max_tokens,
            **kwargs,
        )

    async def ainvoke(
        self,
        prompt: str,
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:
        return self(
            prompt,
            n=n,
            prefix=prefix,
            append=append,
            min_tokens=min_tokens,
            max_tokens=max_tokens,
            **kwargs,
        )

    def batch(
        self,
        prompts: List[str],
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:
        return self(
            prompts,
            n=n,
            prefix=prefix,
            append=append,
            min_tokens=min_tokens,
            max_tokens=max_tokens,
            **kwargs,
        )

    async def abatch(
        self,
        prompts: List[str],
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:
        return self(
            prompts,
            n=n,
            prefix=prefix,
            append=append,
            min_tokens=min_tokens,
            max_tokens=max_tokens,
            **kwargs,
        )

    @staticmethod
    def render_template(
        *, template: str, template_values: Union[List[Dict[str, Any]], Dict[str, Any]]
    ) -> List[str]:
        """
        Substitutes placeholders in a template string with values from a dictionary or
        a list of dictionaries.

        Args:
            template (str): The template string containing placeholders in the format
                '{key}'.
            template_values (Union[List[Dict[str, Any]], Dict[str, Any]]): A dictionary
                or list of dictionaries where keys match the placeholders in the
                template, and values are the substitutions.

        Returns:
            List[str]: A list of rendered template strings with placeholders substituted
            by their corresponding values.

        Raises:
            ValueError: If a substitution pattern in 'template' is not found in
                'template_values', or if a key in 'template_values' is not found in
                'template'.
        """

        # Validate and normalize the template_values input
        if isinstance(template_values, Dict):
            template_values = [template_values]

        # Extract all substitution patterns from the template (placeholders in the form '{key}')
        template_keys = set(re.findall(r"{(.*?)}", template))
        rendered_templates = []

        for v_index, cur_template_value in enumerate(template_values):
            # Check for missing template keys in the current value dictionary
            missing_keys = template_keys - set(cur_template_value.keys())
            if missing_keys:
                raise ValueError(
                    f"Missing keys {missing_keys} in template_values at index {v_index}"
                )

            # Check for extra keys in the current value dictionary that are not used in the template
            extra_keys = set(cur_template_value.keys()) - template_keys
            if extra_keys:
                raise ValueError(
                    f"Extra keys {extra_keys} in template_values at index {v_index} not found in template"
                )

            # Render the template by replacing placeholders with their corresponding values
            rendered_template = template
            for key, value in cur_template_value.items():
                placeholder = f"{{{key}}}"
                rendered_template = rendered_template.replace(placeholder, str(value))

            rendered_templates.append(rendered_template.strip())

        return rendered_templates


class LLMSimulatorLite(LLMSimulatorBase):
    """
    Simple LLM simulator with minimal external dependencies that generates a fixed
    response for each prompt
    """

    def __init__(self, unique: bool = True, chars_per_token: int = 3):

        # When true, add a unique element to each prompt. This is useful when testing
        # applications that use set-style collections as part of their algorithm.
        self.unique = unique

        # When constructing responses of a desired length, this is the calue that is
        # used to estimate the number of tokens per character.
        self.chars_per_token = chars_per_token

    def _estimate_tokens(self, text: str) -> int:
        return math.ceil(len(text) / self.chars_per_token)

    def __call__(
        self,
        prompts: Union[List[str], str],
        *,
        n: int = 1,
        prefix: str = "GEN",
        append: str | None = None,
        min_tokens: int | None = None,
        max_tokens: int | None = None,
        **kwargs,
    ) -> List[str | None]:

        if isinstance(prompts, str):
            prompts = [prompts]

        # Generate a fixed response for each prompt, assume 3 characters per token
        if min_tokens is not None and max_tokens is not None:
            target_tokens = random.randint(min_tokens, max_tokens)
        else:
            target_tokens = 100

        derived_append = append if append is not None else ""

        prefix_tokens = self._estimate_tokens(prefix)
        derived_append_tokens = self._estimate_tokens(derived_append)

        # Generate response strings for each prompt response value
        responses: List[str | None] = []
        for _ in range(n * len(prompts)):

            # Generate a unique UUID for each response if required
            if self.unique:
                derived_uuid = str(uuid4()).replace("-", "")
            else:
                derived_uuid = ""

            # Estimate the number of tokens for each part of the response
            uuid_tokens = self._estimate_tokens(derived_uuid)

            # Calculate how many filler tokens we need to generate
            filler_tokens = target_tokens - (
                uuid_tokens + prefix_tokens + derived_append_tokens
            )

            # Calculate how many filler characters we need to generate, note the -3 is
            # estimating for the shitespace in between response template fields below.
            filler_chars = math.ceil(filler_tokens * self.chars_per_token) - 3

            filler = "A" * filler_chars

            cur_response = f"{prefix},{derived_uuid},{filler},{derived_append}".strip()
            responses.append(cur_response)

        return responses

print("OK")

OK


In [2]:
"""Evaluate MK2"""


import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone
from typing import Dict, List, Tuple

import numpy as np
from scipy.stats import truncnorm

from sandbox.llm_simulator import LLMSimulatorBase, LLMSimulatorLite


class Evaluate:
    """
    MK2 sandbox implementation of the TextEvolve Evaluate function.

    Technical paper: https://bit.ly/3MoNl7A

    Note:
        Changes from MK1

        * Fixed a bug in the normalized score calculation
        * Added parallelized batch evaluation support
        * Added simulated memory selection method
        * Added debate history selection
        * Integrated the LLMSimulator for generating debate turn prompts
        * Performed code quality checks and improvements
        * Added minor documentation improvements
        * Added implementation notes where appropriate
        * Removed pointless debugging print statements

    """

    def __init__(
        self,
        a: List[str],
        w: np.ndarray,
        r: int,
        c: float,
        l: int,  # noqa: E741
        llm_sim: LLMSimulatorBase | None = None,
    ):
        """
        Initialize the evaluator with the given configuration settings.

        Args:
            a (List[str]): List of debater agents.
            w (np.ndarray): Weight vector for score components.
            r (int): Number of debate rounds.
            c (float): Convergence threshold for early stopping.
            l (int): Debate history parameter (currently a placeholder).
        """
        self.a = a
        self.w = w
        self.r = r
        self.c = c
        self.l = l  # noqa: E741

        # Provide an instance of LLMSimulator, this is used to simulate calling the LLM
        # for demonstrations, unit testing, debugging
        self.llm_sim = llm_sim if llm_sim is not None else LLMSimulatorLite()

    # pylint: disable=unused-argument
    def _debate_turn(
        self,
        x: str,
        y: List[str],
        xi: List[str],
        m_i: List[str],
        i: int,
        j: int,
        k: int,
        round_num: int,
        x_name: str = "input",
        y_name: str = "output",
    ) -> np.ndarray:
        """
        Simulate the LLM scoring process for a debate turn using a truncated normal
        distribution. This simulation controls the coefficient of variation (CV) to
        mimic agents reaching consensus over time.

        Note:
            Actual implementations of this function should check to ensure the
            score values returned from the LLM fall within the 0.0 to 10.0 range.
            Problematic responses can either be retried, dropped, or a scaling heuristic
            can be applied to bring them into the valid range.

        Note:
            Actual implementation should consider error handling and retey logic for
            failures, in particular parse errors or scores that are out of bounds.

        Args:
            x (str): The input context in natural language.
            y (List[str]): List of candidate responses.
            xi (List[str]): Debate history.
            m_i (List[str]): Memories for the current agent.
            i (int): Agent identifier.
            j (int): Number of candidate responses.
            k (int): Number of score components.
            round_num (int): The current round number.
            x_name (str): Name of the input context, used to build the debate turn
            prompt
            y_name (str): Name of the candidate responses, used to build the debate turn
            prompt.

        Returns:
            np.ndarray: A j x k matrix with scores between 0.0 and 10.0.
        """

        # Sample debate turn prompt, this will likely need to be optimized for your use
        # case. The instructions are tuned to be flexible enough to evaluate a broad
        # range of x and y variations and agent personas, while pinning the agent to the
        # evaluation criteria to be externally weighted by confidence, relevancy,
        # accuracy, completness, timeliness, and overall score components.
        prompt_template = "\n".join(
            [
                "Your are the {role}. {role_description}",
                "",
                "You are participating in a multi-agent debate to evaluate the {x_name} against each {y_name} below.",
                "",
                "----- BEGIN {x_name_upper} -----",
                "{x}",
                "----- END {x_name_upper} -----",
                "",
                "----- BEGIN {y_name_upper} TO EVALUATE -----",
                "{y}",
                "----- END {y_name_upper} TO EVALUATE -----",
                "",
                "----- BEGIN DEBATE HISTORY -----",
                "{xi}",
                "----- END DEBATE HISTORY -----",
                "",
                "As the {role}, you recall the following memories relevant to this debate:",
                "",
                "----- BEGIN {role_upper} MEMORIES -----",
                "{m_i}",
                "----- END {role_upper} MEMORIES -----",
                "",
                "Your tasks as the {role}, is to evaluate each {y_name} and contribute to the debate.",
                "Consider the following for each {y_name}:",
                "1. As needed, address previous points raised in the debate as they pertain to your evaluation",
                "2. Analyze the relevance and accuracy for each {y_name}",
                "3. Analyze the completeness and depth for each {y_name}",
                "4. Any potential limitations for each {y_name}",
                "5. Analyze any potential limitations for each {y_name}",
                "6. Analyze any timeliness considerations for each {y_name}, is it currently: {current_time}",
                "7. Missing information and/or context for each {y_name}",
                "",
                "Provide your evaluation in a clear, to-the-point, and concise manner, staying true to your role.",
                "",
                "You must evaluate all {y_count} {y_name}.",
            ]
        )

        agent_profiles: Dict[str, str] = {
            "Critic": ""
            "You are a critical thinker who analyzes responses for flaws and "
            "inconsistencies. You question assumptions, point out logical "
            "fallacies, and identify areas where the responses may be lacking or "
            "misleading.",
            "Supporter": ""
            "You are an advocate who looks for strengths and positive aspects in the "
            "responses. Your highlight the merits of each response, explain "
            "potential benefits, and defend good ideas against criticism.",
            "Neutral Observer": ""
            "You are an impartial observer who balances different viewpoints and "
            "provides objective analysis. You consider all perspectives, "
            "weigh the pros and cons, and offer a balanced evaluation of the "
            "responses.",
        }

        # For the simulation, randomly select an agent role
        role, role_description = random.choice(list(agent_profiles.items()))

        rendered_prompt = self.llm_sim.render_template(
            template=prompt_template,
            template_values={
                "role": role,
                "role_upper": role.upper(),
                "role_description": role_description,
                "x": x,
                "x_name": x_name,
                "x_name_upper": x_name.upper(),
                "y": "\n\n".join(
                    [
                        f"{y_name} {y_index+1}:\n{y_value}"
                        for y_index, y_value in enumerate(y)
                    ]
                ),
                "y_name": y_name,
                "y_name_upper": y_name.upper(),
                "y_count": len(y),
                "xi": (
                    "\n".join(xi) if len(xi) > 0 else "(No debate history, first round)"
                ),
                "m_i": (
                    "\n".join(m_i)
                    if len(m_i) > 0
                    else "(No external memories recalled)"
                ),
                "current_time": datetime.now(timezone.utc).strftime(
                    "%Y-%m-%dT%H:%M:%SZ"
                ),
            },
        )

        self.llm_sim(
            rendered_prompt,
            n=1,
            prefix="EVALUATE_TURN",
            min_tokens=200,
            max_tokens=1500,
            temperature=1.0,
            top_p=1.0,
        )

        # Simulation configuration: The Initial coefficient of variation for the first
        # round.
        initial_cv: float = 0.2

        # Simulation configuration: Percentage by which CV decreases each round.
        cv_decrease_percent: float = 10.0

        # Generate mean scores for each candidate response and score component
        # The mean is randomly chosen between 4.5 and 5.5 to center the scores around
        # the middle of the 0-10 range
        mean_score = np.random.uniform(4.5, 6.5, (j, k))  # Shape: (j, k)

        # Compute the current coefficient of variation (CV) for this round
        # CV decreases with each round by the specified percentage to simulate agents
        # reaching consensus
        current_cv = initial_cv * (1 - cv_decrease_percent / 100.0) ** round_num

        # Standard deviation is calculated as a proportion of the mean score, based on
        # the current CV
        std_dev = mean_score * current_cv  # Shape: (j, k)

        # Set the lower and upper bounds for the scores to ensure they stay within the
        # valid range [0.0, 10.0]
        lower, upper = 0.0, 10.0

        # Generate scores using a truncated normal distribution The scores are generated
        # such that they fall within the [0.0, 10.0] range, following a normal
        # distribution centered on mean_score with std_dev
        scores = truncnorm(
            (lower - mean_score) / std_dev,  # Lower bound in standardized units
            (upper - mean_score) / std_dev,  # Upper bound in standardized units
            loc=mean_score,  # Mean of the distribution
            scale=std_dev,  # Standard deviation of the distribution
        ).rvs()  # Generate random variates

        return scores

    @staticmethod
    def _compute_cv(matrix: np.ndarray) -> float:
        """
        Compute the coefficient of variation (CV) for a given score matrix.

        Args:
            matrix (np.ndarray): A matrix of scores (i x j x k) from which CV is
            computed.

        Returns:
            float: The coefficient of variation of the scores.
        """
        return np.std(matrix) / np.mean(matrix)

    def evaluate_batch(
        self,
        batch: List[Tuple[str, List[str]]],
        x_name: str = "input",
        y_name: str = "output",
        workers: int = 10,
        thread_name_prefix: str = "evaluate_pool",
    ) -> List[Tuple[np.ndarray, List[str]] | None]:
        """
        Evaluates a batch of records in parallel using multiple worker threads.

        This method will always preserve the order of the input batch when returning.

        Args:
            batch (List[Tuple[str, List[str]]]): The batch of records to evaluate. Each
                record is a tuple containing an input string and a list of strings. workers
                (int, optional): The number of worker threads to use. Defaults to 10.
            thread_name_prefix (str, optional): The prefix for the worker thread names.
                Defaults to "evaluate_pool".
            workers (int, optional): The number of worker threads to use. Defaults to 10.
            x_name (str): Name of the input context, used to build the debate turn
                prompt.
            y_name (str): Name of the candidate responses, used to build the debate turn
                prompt.

        Returns:
            List[Tuple[np.ndarray, List[str]] | None]: A list of tuples containing the
            evaluation result (S) and the debate for each record in the batch. If an
            error occurs during evaluation, the tuple will contain None values.

        Raises:
            None

        Examples:
            # Example usage
            batch = [
                ("input1", ["string1", "string2"]),
                ("input2", ["string3", "string4"]),
                ("input3", ["string5", "string6"]),
            ]
            results = evaluate_batch(batch, workers=5)

        """
        pool = ThreadPoolExecutor(
            thread_name_prefix=thread_name_prefix, max_workers=workers
        )

        def worker(
            key: int,
            x: str,
            y: List[str],
            x_name: str,
            y_name: str,
        ) -> Tuple[int, np.ndarray, List[str]] | Tuple[int, None, None]:
            """
            Worker thread inner function that evaluates a batch record.

            Args:
                key (int): The key for the batch record.
                x (str): The input string.
                y (List[str]): The list of strings.


            Returns:
                Tuple[int, np.ndarray, List[str]] | Tuple[int, None, None]: A tuple
                containing the key, the evaluation result (S), and the debate. If an
                error occurs during evaluation, returns a tuple with None values.
            """
            try:
                S, debate = self.evaluate(x=x, y=y, x_name=x_name, y_name=y_name)
                return (key, S, debate)
            except:

                LOG.exception("Error evaluating batch record: %i", key)
                return (key, None, None)

        try:
            # Submit worker pool tasks
            futures = []
            for i, batch_record in enumerate(batch):
                x, y = batch_record
                future = pool.submit(worker, i, x, y, x_name, y_name)
                futures.append(future)

            # Get results in the order of the batch
            keyed_results = []
            for future in as_completed(futures):
                result = future.result()
                keyed_results.append(result)

            # Sort the results based on the original order of the batch
            keyed_results = sorted(keyed_results, key=lambda x: x[0])

            # Remove key from results since it was only needed to maintain order of the
            # final result.
            eval_results: List[Tuple[np.ndarray, List[str]] | None] = [None] * len(
                batch
            )
            for cur_keyed_result in keyed_results:

                key = cur_keyed_result[0]
                S = cur_keyed_result[1]
                debate = cur_keyed_result[2]

                # Skip 'None' values, these are errors
                if S is None or debate is None:
                    continue

                # Else, store the result to be returned
                eval_results[key] = (
                    S,
                    debate,
                )

            return eval_results
        finally:
            pool.shutdown(wait=False, cancel_futures=True)

    def _m(self, i: int, x: str, y: List[str], xi: List[str]) -> List[str]:
        """Placeholder method that simulates retrieving memories from an agent.

        Args:
            i (int): The agent index.
            x (str): Input context in natural language.
            y (List[str]): List of candidate responses.
            xi (List[str]): Debate history, use by the memory retrievela algorithm.

        Returns:
            List[str]: A list of retrieved memories for the agent.
        """

        return [f"Memory {x+1}" for x in range(10)]

    def evaluate(
        self,
        x: str,
        y: List[str],
        x_name: str = "input",
        y_name: str = "output",
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Evaluate the candidate responses using the TextEvolve evaluation function.

        Args:
            x (str): Input context in natural language.
            y (List[str]): List of candidate responses.
            x_name (str): Name of the input context, used to build the debate turn
            prompt.
            y_name (str): Name of the candidate responses, used to build the debate turn
            prompt.

        Returns:
            Tuple[np.ndarray, List[str]]: Tuple containing the scoring tensor S with
            dimensions (r x i x j x k) and simulated debate history.
        """
        j = len(y)  # Number of candidate responses
        k = len(self.w)  # Number of score components
        i = len(self.a)  # Number of agents
        S = np.zeros((self.r, i, j, k))  # Initialize scoring tensor

        debate: List[str] = []

        for round_num in range(self.r):
            for agent_num in range(i):

                # Placeholder for debate history (ξ)
                xi = debate[-self.l * round_num :]

                # Retrieve memories for the current agent
                m_i = self._m(i=agent_num, x=x, y=y, xi=xi)

                S[round_num, agent_num] = self._debate_turn(
                    x=x,
                    y=y,
                    xi=xi,
                    m_i=m_i,
                    i=agent_num,
                    j=j,
                    k=k,
                    round_num=round_num,
                    x_name=x_name,
                    y_name=y_name,
                )

                # Simulate a debate history entry for this turn
                ts = int(datetime.now().timestamp() * 1000) % 100000
                debate.append(
                    f"{ts}: round {round_num}, agent {agent_num}, response candidates "
                    f"1 to {j} {{DEBATE_TURN}}"
                )

            # After each round, compute CV and check for early stopping
            round_cv = self._compute_cv(matrix=S[round_num])

            if round_cv <= self.c:
                LOG.info(
                    "Early stopping triggered at Round %i (CV <= %f)",
                    round_num + 1,
                    self.c,
                )

                S = S[: round_num + 1]  # Truncate the tensor to the completed rounds
                break

        return (S, debate)

    def compute_normalized_scores(self, S: np.ndarray) -> np.ndarray:
        """
        Compute the normalized scores for each response candidate.

        Args:
            S (np.ndarray): The scoring tensor with dimensions (r x i x j x k).

        Returns:
            np.ndarray: Normalized score vector for each candidate response.
        """
        return np.sum(S * self.w, axis=(0, 1, 3)) / (
            S.shape[0] * S.shape[1] * S.shape[3]
        )

    @staticmethod
    def compute_softmax_scores(s_norm: np.ndarray) -> np.ndarray:
        """
        Compute the softmax scores for each response candidate.

        Args:
            s_norm (np.ndarray): Normalized score vector.

        Returns:
            np.ndarray: Softmax score vector representing probabilities.
        """
        e_x = np.exp(s_norm - np.max(s_norm))  # Subtract max for numerical stability
        return e_x / e_x.sum(axis=0)

    def select_best_candidate(
        self, S: np.ndarray, y: List[str], probabilistic: bool = False
    ) -> str:
        """
        Select the best response candidate based on normalized or softmax scores.

        Args:
            S (np.ndarray): The scoring tensor with dimensions (r x i x j x k).
            y (List[str]): List of candidate responses.
            probabilistic (bool): If True, selects based on softmax scores; otherwise,
            based on normalized scores.

        Returns:
            str: The selected response candidate.
        """
        # Compute normalized scores
        s_norm = self.compute_normalized_scores(S)

        if probabilistic:
            # Compute softmax scores
            s_phi = self.compute_softmax_scores(s_norm)

            # Select the best response candidate probabilistically using softmax scores
            best_index = np.random.choice(np.arange(len(y)), p=s_phi)
        else:

            # Select the best response candidate using normalized scores
            best_index = np.argmax(s_norm)

        return y[best_index]

print("OK")

OK


### Calibrate Code

In [4]:
"""Sandbox implementation of the TextEvolve Calibrate function."""

import logging
import random
import re
from typing import Any, Dict, List, Literal, Set, Tuple, cast

import numpy as np

from sandbox.llm_simulator import LLMSimulatorBase, LLMSimulatorLite
from sandbox.text_evolve.evaluate import Evaluate

LOG = logging.getLogger(__name__)


class CalibrateException(Exception):
    """
    TextEvolve Calibrate error.
    """



class Calibrate:
    """
    TextEvolve Calibrate

    This is a demonstration implementation of the TextEvolve Calibrate algorithm where
    several sections of the code are stubbed out for simulation purposes. The algorithm
    is designed to optimize a prompt template for a language model by iteratively
    expanding and scoring the prompt template against a calibration dataset.

    Production implementations will need to modify the following items:

    * Remove the llm_sim parameter from the constructor
    * Implement _llm_generate, _llm_gradients, and _llm_sigma, to invoke an actual LLM


    Requires Python 3.11+
    """

    def __init__(
        self,
        b: int,
        r: int,
        minibatch_exp_len: int,
        minibatch_sel_len: int,
        evaluator: Evaluate,
        grad_depth: int = 10,
        grad_debate_rounds: int = 2,
        mc_successors: int = 10,
        select_method: Literal["fast", "ucb"] = "fast",
        early_stopping: bool = True,
        llm_sim: LLMSimulatorBase | None = None,
    ):
        """
        Initialize the TextEvolve Calibrate optimizer with the specified
        hyper-parameters.

        Args:
            b (int): Beam width, determining how many candidate prompts to keep at each
                level.
            r (int): Search depth, or how many rounds of optimization to perform.
            minibatch_exp_len (int): Number of calibration data samples for prompt
                expansion.
            minibatch_sel_len (int): Number of calibration data samples for scoring
                expanded prompts.
            evaluator (Evaluate): An instance of Evaluate used to evaluate and score
                candidates.
            grad_depth (int): Number of suggestions the LLM generates when constructing
                gradients.
            grad_debate_rounds (int): Number of debate rounds to consider when
                generating gradients.
            mc_successors (int): Number of Monte Carlo successors generated from the
                improved template.
            select_method (Literal["fast", "ucb"]): Method for selecting candidates
                ('fast' or 'ucb').
            early_stopping (bool): If True, stops early if no new beams are discovered
                in a round.
            llm_sim (Optional[LLMSimulatorBase]): Instance of LLMSimulator for
                simulation or testing.
        """

        self.b = b  # Beam width
        self.r = r  # Search depth

        # Controls how many calibration data records are sampled for each prompt
        # expansion. This value will vary depending on the backend LLM that is used
        # where more sophisticated LLMs with larger token limits can handle larger
        # mini-batches. Notice that large mini-batches will require more tokens and more
        # sophisticated reasoning during scoring.
        #
        # Limitations: Too large a value will burden Evaluate with too many records to score
        self.minibatch_exp_len = minibatch_exp_len

        # Controls how many calibration records are samples when deriving the scores for
        # expanded prompts
        #
        # Limitations: Too large a value will burden Evaluate with too many records to score
        self.minibatch_sel_len = minibatch_sel_len

        # Controls the number of suggestions the LLM is asked to generate when
        # constructing gradients. More suggestions will allow for more sophisticated
        # improvements per beam search iteration.
        self.grad_depth = grad_depth

        # Controls the number of debate rounds to consider when generating gradients. A
        # round is a full debate cycle where each agent takes a turn. For example, if
        # there are 3 agents, 1 debate round would include a response from all 3 agents.
        self.grad_debate_rounds = grad_debate_rounds

        # Once a new prompt template has been generated from gradients, this value
        # controls how many additional monte-carlo successors will be generated from the
        # improved template.
        self.mc_successors = mc_successors

        # A pre-configured instance if the TextEvolveEvaluator class
        self.evaluator = evaluator

        # Select method
        self.select_method = select_method

        # When early stopping is enabled, the number of new beams discovered will be
        # checked after each round. If no new beams are detected the remaining rounds
        # will be skipped.
        self.early_stopping = early_stopping

        # Provide an instance of LLMSimulator, this is used to simulate calling the LLM
        # for demonstrations, unit testing, debugging
        self.llm_sim = llm_sim if llm_sim is not None else LLMSimulatorLite()

    @staticmethod
    def _top_k_beams(*, beams: Dict[str, float], k: int) -> Dict[str, float]:
        """
        Selects the highest scoring beams.

        Args:
            beams (Dict[str, float]): A dictionary containing the beam text (key) and
                score (value).
            k (int): The number of highest scoring beams to select.

        Returns:
            Dict[str, float]: A dictionary containing the top k key-value pairs.
        """

        if k <= 0:
            raise ValueError("k must be positive")

        # If k is greater than or equal to the length of data, just return everything.
        if k >= len(beams):
            return beams

        # Sort beams by score and select the last k items, these represent the highest
        # scores
        top_k_keys: List[str] = sorted(beams, key=beams.get)[-k:]  # type: ignore

        # Create a new dictionary with the top k items
        selected: Dict[str, float] = {}
        for cur_k in top_k_keys:
            selected[cur_k] = beams[cur_k]

        return selected

    def _llm_generate(
        self,
        templates: List[Dict[str, List[Dict[str, Any]]]],
        prefix: str,
    ) -> List[str]:
        """
        Simulates calling an LLM for a batch of test generation templates.

        Args:
            templates (List[Dict[str, List[Dict[str, Any]]]]): A list where each item is a
                dictionary mapping a prompt template (str) to a list of template values
                (List[Dict[str, Any]]). Each template will be rendered with its corresponding
                template values.
            prefix (str): A prefix string used in the LLM simulation.

        Returns:
            List[str]: A list of generated strings from the LLM simulation.
        """

        # Generation model parameters, used when generating samples from a prompt
        # template and calibration record. Recommended settings are {"temperature": 1.0,
        # "top_p": 1.0}; additionally maximum tokens will be use case specific.

        # Render prompt strings for simulated batch inferencing
        rendered_prompts: List[str] = []
        for template in templates:
            for key, values in template.items():
                rendered_prompts.extend(
                    self.llm_sim.render_template(template=key, template_values=values)
                )

        # Simulate LLM batch inferencing
        generations = self.llm_sim(
            rendered_prompts,
            n=1,
            prefix=prefix,
        )

        # Filter out any None values from the generation results and ensure we still
        # have a result. Alternatively, we could implement retry logic to handle
        # transient LLM errors.
        generations_valid = [x for x in generations if x is not None]
        if len(generations_valid) == 0:
            raise CalibrateException("LLM was unable to generate results")

        return generations_valid

    def _llm_gradients(self, t: str, mean_score: float, errors: List[str]) -> List[str]:
        """
        Generates gradient instructions to improve the prompt template based on
        evaluation feedback.

        Args:
            t (str): The current prompt template.
            mean_score (float): The mean score obtained from the evaluation of the
                prompt.
            errors (List[str]): A list of error messages or debate history from the
                evaluation.

        Returns:
            List[str]: A list of gradient instructions suggested by the LLM to improve
                the prompt template.
        """

        # Build the debate history string from 'errors' returned by the
        # evaluate step. Here, we trim the full debate history to the last
        # 'grad_debate_rounds' rounds.
        debate_turns = errors[-self.grad_debate_rounds * len(self.evaluator.a) :]
        debate = "\n".join(debate_turns)

        # Compute 'gradients', this is analogous to the back-propagation step when
        # training neural networks. I deliberately decided to compute 'g' together in a
        # single call to the LLM since gradients must take into account the entire score
        # on the entire minibatch.
        #
        # Insight: the paper did not include the actual response in the gradient
        # creation prompt so I left it out as well.

        prompt_template = "\n".join(
            [
                "I need help improving a text-based prompt template for a language "
                "model. The current prompt template is:",
                "",
                "----- BEGIN PROMPT TEMPLATE -----",
                "{t}",
                "----- END PROMPT TEMPLATE -----",
                "",
                "The block below contains a debate between AI agents who are deciding how "
                "to score an output generated by the LLM prompt below:",
                "",
                "----- BEGIN DEBATE -----",
                "{debate}",
                "----- END DEBATE -----",
                "",
                "The agents aligned on an average score of {score} out of 10 after the "
                "debate.",
                "",
                "Your task is to generate a terse and highly specific list of "
                "instructions to improve the average score by addressing problems "
                "raised in the debate history.",
                "",
                "Your list of instructions may contain up to {grad_depth} "
                "instructions.",
            ]
        )

        rendered_prompt = self.llm_sim.render_template(
            template=prompt_template,
            template_values={
                "t": t,
                "debate": debate,
                "score": mean_score,
                "grad_depth": self.grad_depth,
            },
        )

        # Call LLM to generate gradients from Evaluate feedback
        g = self.llm_sim(
            rendered_prompt,
            n=1,
            prefix="GRADIENT",
        )

        if g[0] is None:
            raise CalibrateException("LLM was unable to generate gradients")

        # For simulation purposes, we will expand 'g' to the number of requested
        # gradients
        derived_g = cast(List[str], [g[0]] * self.grad_depth)

        return derived_g

    def _llm_sigma(self, t: str, g: List[str]) -> List[str]:
        """
        Generates improved prompt templates by applying gradient instructions to the
        current template.

        Args:
            t (str): The current prompt template.
            g (List[str]): A list of gradient instructions to apply.

        Returns:
            List[str]: A list of improved prompt templates generated by the LLM.
        """

        prompt_template = "\n".join(
            [
                "Consider the following natural language prompt template:",
                "",
                "----- BEGIN PROMPT TEMPLATE -----",
                "{t}",
                "----- END PROMPT TEMPLATE -----",
                "",
                "Your task is to apply the following suggestions to improve the template:",
                "{g}",
                "",
                "Note that the prompt template contains Python f-string style substitution "
                "placeholders enclosed in curly braces, your improved prompt must preserve "
                "all template placeholders.",
            ]
        )

        rendered_prompt = self.llm_sim.render_template(
            template=prompt_template,
            template_values={
                "t": t,
                "g": g,
            },
        )

        # For the simulation, we will extract the template fields from the original
        # prompt template and just append to the end of the generation so the output will
        # pass the check below
        template_keys = set(re.findall(r"{(.*?)}", t))
        template_key_str = " ".join([f"{{{x}}}" for x in template_keys])

        # Update the original prompt template with the 'gradients'. In the
        # paper there is an additional step to generate additional monte-carlo
        # successors from p', in this implementation we accomplish this by
        # increasing the number of generations requested from the LLM.
        t_expanded = self.llm_sim(
            rendered_prompt,
            n=self.mc_successors,
            prefix="EXPANDED",
            append=template_key_str,
        )

        # Ensure that the expanded templates contain all the original template keys.
        template_patterns = [f"{{{x}}}" for x in template_keys]
        t_expand_validated: List[str] = []
        for t_exp in t_expanded:
            if t_exp is None:
                continue

            # Verify that all patterns in template_patterns are in t_exp
            if not all(re.search(pattern, t_exp) for pattern in template_patterns):
                LOG.warning(
                    "Expanded prompt does not contain all original template keys"
                )
                continue

            t_expand_validated.append(t_exp)

        # If the validated list of expanded prompts is less than the requested number of
        # monte-carlo successors, raise an exception. Alternatively a threshold can be
        # checked for fault tolerance.
        if len(t_expand_validated) < self.mc_successors:
            raise CalibrateException(
                f"Only {len(t_expand_validated)} valid expanded prompts generated; expected {self.mc_successors}"
            )

        return t_expand_validated

    def _score(
        self, *, batch: List[Tuple[str, List[str]]]
    ) -> List[Tuple[float, List[str]] | None]:
        """
        Computes evaluation scores for a batch of prompt templates and their generated
        outputs.

        Args:
            batch (List[Tuple[str, List[str]]]): A list of tuples where each tuple
                contains a prompt template (str) and a list of generated outputs
                (List[str]) for that template.

        Returns:
            List[Tuple[float, List[str]] | None]: A list where each item is either a
                tuple containing the mean score (float) and a list of errors
                (List[str]) for each prompt, or None if evaluation failed.
        """

        eval_results = self.evaluator.evaluate_batch(batch=batch)

        scores: List[Tuple[float, List[str]] | None] = [None] * len(batch)
        for i, cur_eval_result in enumerate(eval_results):
            if cur_eval_result is None:
                continue

            S, errors = cur_eval_result

            score = self.evaluator.compute_normalized_scores(S)
            mean_score = np.mean(score)

            scores[i] = (mean_score, errors)

        return scores

    def _expand(self, t: str, d_cal: List[Dict[str, Any]]) -> List[str]:
        """
        Expands the given prompt template by generating multiple improved templates
          using calibration data.

        Args:
            t (str): The prompt template to be expanded.
            d_cal (List[Dict[str, Any]]): The calibration data used for expanding the
                prompt.

        Returns:
            List[str]: A list of expanded prompt templates generated from the original
                template.
        """

        # Sample a minibatch of calibration data
        minibatch = random.sample(d_cal, self.minibatch_exp_len)

        # For the current beam, generate responses from the LLM
        generations = self._llm_generate(
            templates=[{t: minibatch}], prefix="GEN_EXPAND"
        )

        if len(generations) != len(minibatch):
            raise ValueError(
                "Error generating prompt template expansions. Number of generations "
                "does not match the minibatch size."
            )

        # Compute the 'loss' (score) for the expanded prompt template generations
        scores = self._score(batch=[(t, generations)])

        if len(scores) != 1 or scores[0] is None:
            raise ValueError("Error scoring prompt generation during expand")

        # pylint: disable=unpacking-non-sequence
        mean_score, errors = scores[0]

        # Generate gradients from the evaluation feedback by investigating what went
        # wrong given the debate history (errors).
        g = self._llm_gradients(t=t, mean_score=mean_score, errors=errors)

        t_expanded = self._llm_sigma(t=t, g=g)

        return t_expanded

    def _select_ucb(self, C: Set[str], d_cal: List[Dict[str, Any]]) -> Dict[str, float]:
        raise NotImplementedError("UCB selection not implemented yet")

    def _select_fast(
        self, C: Set[str], d_cal: List[Dict[str, Any]]
    ) -> Dict[str, float]:
        """
        Selects the best prompt templates from candidates using the UCB
        (Upper Confidence Bound) method.

        Args:
            C (Set[str]): A set of candidate prompt templates.
            d_cal (List[Dict[str, Any]]): The calibration data for scoring the
                candidates.

        Returns:
            Dict[str, float]: A dictionary mapping prompt templates to their
                evaluation scores.

        Raises:
            NotImplementedError: This method is not implemented yet.
        """

        # Convert C into an ordered list so we can properly re-assemble the order of
        # batch inference generations
        C_ordered = list(C)

        # Build generation units of work for batch generation
        gen_batch = []
        for c in C_ordered:
            minibatch = random.sample(d_cal, self.minibatch_sel_len)
            gen_batch.append({c: minibatch})

        generations = self._llm_generate(templates=gen_batch, prefix="GEN_SELECT")

        # Match the batch generations to the original prompt template
        score_batch: List[Tuple[str, List[str]]] = []
        for generation_index, generation in enumerate(generations):

            score_batch_index = generation_index // self.minibatch_sel_len

            if generation_index % self.minibatch_sel_len == 0:
                score_batch.append((C_ordered[score_batch_index], []))

            score_batch[score_batch_index][1].append(generation)

        scores = self._score(batch=score_batch)

        # Flatten the scores list, note that errors returned by the scoring process (if
        # any) are re-mapped to 0.0 and should be filtered out when we build the new
        # beam below. Note that scores are returned in the same order as the list of
        # inputs. Finally we are dropping the gradients since they are not being used
        # during selection.
        scores_flat = [x[0] if x is not None else 0.0 for x in scores]

        # Re-assemble the scored components into the data structure that represents our
        # selected beam (B_prime). An important design element here is that we are not
        # filtering out any scores since the outer calibration method will combine
        # B_prime with the current beam (B) and then select the best scores to ensure
        # the the highest scores globally are kept in the search.
        B_prime = dict(
            zip(
                [x[0] for x in score_batch],
                scores_flat,
            )
        )

        return B_prime

    def calibrate(self, t_0: str, d_cal: List[Dict[str, Any]]) -> str:
        """
        Executes the TextEvolve Calibrate algorithm to calibrate an initial prompt
        template against a set of calibration template values.

        Args:
            t_0 (str): The initial prompt template containing placeholders for the
                calibration dataset.
            d_cal (List[Dict[str, Any]]): The calibration dataset containing template
                parameters.

        Returns:
            str: The highest scoring prompt template after calibration.
        """

        # Initialize beam B_0 with our un-scored initial template
        B: Dict[str, float] = {t_0: 0.0}

        # Iterate over the search depth (r rounds)
        for i in range(self.r):

            # C is a dictionary of expanded candidates for this round
            C: Set[str] = set()

            # Expand each beam candidate, note for logging here we add 0 fill since it
            # is in a loop and will be easier to read if the message lengths line up.
            for B_index, t in enumerate(B):
                t_prime = self._expand(t=t, d_cal=d_cal)
                C.update(t_prime)
                LOG.info(
                    "r=%02i expand(B=%02i): len(t')=%02i, len(C)=%02i",
                    i + 1,
                    B_index,
                    len(t_prime),
                    len(C),
                )

            # Select - there is a deep-learning analogy here with validation.
            if self.select_method.lower() == "fast":

                B_prime = self._select_fast(C=C, d_cal=d_cal)

            elif self.select_method.lower() == "ucb":
                B_prime = self._select_ucb(C=C, d_cal=d_cal)

            else:
                raise ValueError(f"Invalid selection method: {self.select_method}")

            # Insight - common operations of the select step happen below.

            # Combine the current beams (B) with the selected beams from this round
            # (B_prime) and select the scores up to the configured beam width.
            B_new = Calibrate._top_k_beams(
                beams={
                    **B,
                    **B_prime,
                },
                k=self.b,
            )

            # Count the number of keys from B_new that are not in B; basically this will
            # tell use how many new prompt templates were discovered in this round. This
            # will provide insight into how the algorithm is improving the prompt
            # template and also give data needed to trigger early stopping.
            new_beams = len(set(B_new.keys()) - set(B.keys()))

            B = B_new

            # Generate a helpful log message to give calibration status for each round
            LOG.info(
                "r=%02i select[%s]: len(B')=%i, new beams=%i, len(B)=%i, B mean=%.4f (%s)",
                i + 1,
                self.select_method.lower(),
                len(B_prime),
                new_beams,
                len(B),
                np.array(list(B.values())).mean(),
                ", ".join(f"{val:.4f}" for val in B.values()),
            )

            if self.early_stopping and new_beams == 0:
                LOG.info("Early stopping triggered, no new beams discovered")
                break

        # Select the top beam from the final round
        best_candidate = max(B, key=B.get)  # type: ignore

        return best_candidate

print("OK")

OK


### Demo

In [6]:
import logging

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s',  # Customize the log format
    handlers=[
        logging.StreamHandler()  # Stream the logs to the notebook's output
    ]
)

b = 9
r = 4
minibatch_exp_len = 5
minibatch_sel_len = 6
grad_depth = 10
grad_debate_rounds = 2
mc_successors = 5

# Configure the TextEvolve Evaluate function. Here, we are simulating a multi-agent debate among 
# 3 agents: the critic, the supporter, and the neutral observer 
evaluator = Evaluate(
    a=["Critic", "Supporter", "Neutral Observer"],
    w=np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
    r=2,
    c=0.05,
    l=1,
)

prompt_calibrator = Calibrate(
    b=b,
    r=r,
    evaluator=evaluator,
    minibatch_exp_len=minibatch_exp_len,
    minibatch_sel_len=minibatch_sel_len,
    grad_depth=grad_depth,
    grad_debate_rounds=grad_debate_rounds,
    mc_successors=mc_successors,
    select_method="fast",
)

d_cal = [
    {"document": "report", "topic": "AI"},
    {"document": "essay", "topic": "philosophy"},
    {"document": "article", "topic": "technology"},
    {"document": "post", "topic": "economics"},
    {"document": "review", "topic": "politics"},
    {"document": "paper", "topic": "history"},
    {"document": "guide", "topic": "science"},
    {"document": "letter", "topic": "art"},
    {"document": "memo", "topic": "education"},
    {"document": "summary", "topic": "health"},
    {"document": "proposal", "topic": "business"},
    {"document": "manual", "topic": "literature"},
    {"document": "story", "topic": "environment"},
    {"document": "script", "topic": "culture"},
    {"document": "novel", "topic": "society"},
    {"document": "blog", "topic": "media"},
    {"document": "journal", "topic": "sports"},
    {"document": "thesis", "topic": "entertainment"},
    {"document": "dissertation", "topic": "food"},
    {"document": "book", "topic": "travel"},
]

t_0 = "Write a {document} about {topic}"

optimized_prompt = prompt_calibrator.calibrate(
    t_0=t_0,
    d_cal=d_cal
)

print("")
print(f"Demo optimized prompt: {optimized_prompt}")

2024-09-19 14:15:38,575 - INFO - r=01 expand(B=00): len(t')=05, len(C)=05
2024-09-19 14:15:38,595 - INFO - r=01 select[fast]: len(B')=5, new beams=5, len(B)=6, B mean=4.5985 (0.0000, 5.4957, 5.5326, 5.5649, 5.4375, 5.5604)
2024-09-19 14:15:38,600 - INFO - r=02 expand(B=00): len(t')=05, len(C)=05
2024-09-19 14:15:38,605 - INFO - r=02 expand(B=01): len(t')=05, len(C)=10
2024-09-19 14:15:38,610 - INFO - r=02 expand(B=02): len(t')=05, len(C)=15
2024-09-19 14:15:38,616 - INFO - r=02 expand(B=03): len(t')=05, len(C)=20
2024-09-19 14:15:38,622 - INFO - r=02 expand(B=04): len(t')=05, len(C)=25
2024-09-19 14:15:38,627 - INFO - r=02 expand(B=05): len(t')=05, len(C)=30
2024-09-19 14:15:38,744 - INFO - r=02 select[fast]: len(B')=30, new beams=8, len(B)=9, B mean=5.5993 (5.5610, 5.5628, 5.5649, 5.5711, 5.5901, 5.6002, 5.6396, 5.6456, 5.6583)
2024-09-19 14:15:38,749 - INFO - r=03 expand(B=00): len(t')=05, len(C)=05
2024-09-19 14:15:38,754 - INFO - r=03 expand(B=01): len(t')=05, len(C)=10
2024-09-19 


Demo optimized prompt: EXPANDED,f6fe2dd0969e4c83847509d6dd373415,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,{topic} {document}
